Data Source

https://www.kaggle.com/datasets/luisblanche/covidct

In [ ]:
# pip install tensorflow==2.13.0

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
# import tensorflow.keras.backend as K
#tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)


In [ ]:
import tensorflow as tf
def limitgpu(maxmem):
	gpus = tf.config.list_physical_devices('GPU')
	if gpus:
		# Restrict TensorFlow to only allocate a fraction of GPU memory
		try:
			for gpu in gpus:
				tf.config.experimental.set_virtual_device_configuration(gpu,
						[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=maxmem)])
		except RuntimeError as e:
			# Virtual devices must be set before GPUs have been initialized
			print(e)


# 1.5GB
limitgpu(1024+512) 


In [ ]:
pip show tensorflow

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
!pip install opencv-python==3.4.18.65

In [ ]:
## Importing Relevant Libraries

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)
## Imports libs

In [ ]:
print("OpenCV version:", cv2.__version__)

In [ ]:
pip install lime==0.1.1.29

In [ ]:
pip show lime

## Renaming images data to Specified format

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_merged/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_merged/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

## Data Preprocessing
### Processing images for machine learning to create two-class dataset

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

#### Processing CT_scan Covid-19  CT_scan nonCovid-19 images

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
ct_covid_features_df

In [ ]:
#imshow(ct_covid_features_df.iloc[0].to_numpy().reshape((100,150)))

In [ ]:
ct_noncovid_features_df

In [ ]:
#imshow(ct_noncovid_features_df.iloc[1].to_numpy().reshape((100,150)))

#### Concatenating the two data frame to create a dataset ready for machine learning

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
ct_covid_features_df.shape

In [ ]:
ct_noncovid_features_df.shape

In [ ]:
cvd_imgs_dataset.shape

In [ ]:
#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['output_encode_0']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_.shape))

In [ ]:
#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['output_encode_0']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x__.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y__.shape))

In [ ]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

In [ ]:
type(input_data_x)

In [ ]:
input_data_x.shape

In [ ]:
output_label_y.shape

In [ ]:
## Images of Negative Covid-19
for i in range(10):
    imshow(input_data_x_[i])
    show()
    

In [ ]:
## Images of Positive Covid-19
for i in range(15):
    imshow(input_data_x__[i])
    show()
    

#### visualize first 5 images from the dataset

In [ ]:
for i in range(15):
    imshow(input_data_x[i])
    show()
    

### Feature Scaling

#### **Splitting the dataset into the Training set and Test set**

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_, test_features_, train_labels_, test_labels_ = train_test_split(
    input_data_x_, output_label_y_, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features__, test_features__, train_labels__, test_labels__ = train_test_split(
    input_data_x__, output_label_y__, test_size=.20, random_state=42)

In [ ]:
train_features__.shape

In [ ]:
test_features__.shape

### IMPLEMENTATION OF SHAP

In [ ]:
###VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def VGG16():
    model = Sequential()

    model.add(ZeroPadding2D((1,1), input_shape=(300, 300, 1)))  # Input shape corrected

    # Block 1
    model.add(Conv2D(2, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(2, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(4, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(4, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer="he_normal"))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='inter'))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu', name='fc2'))
    model.add(Dense(2, activation='softmax'))  # Updated to 2 classes for binary classification

    return model



In [ ]:

modelss = VGG16()
modelss.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

modelss.fit(train_features, train_labels, epochs=50, batch_size=16)

In [ ]:
modelss.save('VGG16_model.h5')

In [ ]:
### Implementation of SHAP

In [ ]:
import shap
from shap import initjs

In [ ]:
model_VGG16 = tf.keras.models.load_model('VGG16_model.h5')

In [ ]:
model_VGG16.summary()

In [ ]:
explainVGG16 = shap.GradientExplainer(model_VGG16,train_features)
explainVGG16 

In [ ]:
shap_valuesVGG16 = explainVGG16.shap_values(test_features)

In [ ]:
shap.image_plot(shap_valuesVGG16[:5], test_features[:5])

In [ ]:
### SHAP on Negative test features

In [ ]:
explain_negative = shap.GradientExplainer(model_VGG16,test_features_)
explainVGG16 

In [ ]:
shap_values_negative = explain_negative.shap_values(test_features_)

In [ ]:
### SHAP on Positive test features

In [ ]:
explain_positive = shap.GradientExplainer(model_VGG16,test_features__)
explainVGG16 

In [ ]:
shap_values_positive = explain_positive.shap_values(test_features__)

In [ ]:
import numpy as np
import cv2
import os

def process_and_save_images(images, shap_values, threshold, output_folder):
    """
    Process images by removing non-important features based on SHAP values and save the images.

    Parameters:
    - images: numpy array of shape (num_images, height, width) or (num_images, height, width, channels)
    - shap_values: numpy array of shape (num_images, height, width) or (num_images, height, width, channels)
    - threshold: importance threshold to decide which features are important
    - output_folder: directory to save the processed images
    """

    # Create masks based on SHAP values and threshold
    important_features_mask = np.abs(shap_values) > threshold

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process and save each image
    for idx, (img, mask) in enumerate(zip(images, important_features_mask)):
        # Ensure image and mask are of the same shape
        if img.shape != mask.shape:
            raise ValueError(f"Image shape {img.shape} and mask shape {mask.shape} do not match.")

        # Apply mask to the image
        processed_img = img * mask

        # Ensure the image is within the range 0-255 for saving
        processed_img = processed_img - np.min(processed_img)
        processed_img = (processed_img / np.max(processed_img) * 255).astype(np.uint8)

        # Save the image
        output_file = os.path.join(output_folder, f'processed_image_{idx}.png')
        cv2.imwrite(output_file, processed_img)


In [ ]:
##### Images for only negative images

In [ ]:
shap_negative = shap_values_negative[:, :, :, :, 0]
shap_negative.shape

In [ ]:
threshold = np.percentile(np.abs(shap_negative), 95)
save_path = "dataset_important_features/VGG16/Shap/Negative" 
process_and_save_images(test_features_, shap_negative, threshold, save_path)

In [ ]:
##### Images for only positive images

In [ ]:
shap_positive = shap_values_positive[:, :, :, :, 0]
shap_positive.shape

In [ ]:
threshold_ = np.percentile(np.abs(shap_positive), 95)
save_path = "dataset_important_features/VGG16/Shap/Positive" 
process_and_save_images(test_features_, shap_positive, threshold_, save_path)

In [ ]:
#### Retrain 

In [ ]:
new_path_ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Shap/Positive/'
new_prefix = 'ct_covid'

new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Shap/Negative/'
new_prefix_ = 'non_ct_covid'

In [ ]:
ct_covid_features_df_ =  processImages(new_path_,1)#1--> covid-19 positive
ct_noncovid_features_df_ =  processImages(new_path__,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs_ = [ct_noncovid_features_df_, ct_covid_features_df_]
cvd_imgs_dataset_ = pd.concat(cvd_imgs_)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap_ = cvd_imgs_dataset_.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap_)

In [ ]:
cvd_imgs_dataset_shap_.shape

In [ ]:
input_data_shap_x_ = cvd_imgs_dataset_shap_.iloc[:,:-1].to_numpy().reshape((870,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap_['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap_['class_label'])
cvd_imgs_dataset_shap_
cvd_imgs_dataset_shap_ = pd.get_dummies(cvd_imgs_dataset_shap_, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_shap_y_ = np.array(cvd_imgs_dataset_shap_[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_shap_x_.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_shap_y_.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap_, test_features_shap_, train_labels_shap_, test_labels_shap_ = train_test_split(
    input_data_shap_x_, output_label_shap_y_, test_size=.20, random_state=42)

In [ ]:
modelss = VGG16()
modelss.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

modelss.fit(train_features_shap_, train_labels_shap_, epochs=10, batch_size=16)

In [ ]:
### Applying LIME on Negative

In [ ]:
train_features_[0].shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_VGG16.predict(images)

single_image_neg = test_features_[0] 
single_image_neg.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks_neg = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image_neg,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks_neg.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
import numpy as np
from skimage.io import imsave
from skimage.util import img_as_ubyte
from skimage.filters import gaussian
import os

def apply_lime_mask(images, lime_masks, method='blur', blur_sigma=5, save=False, save_path='dataset_lime_filtered'):
    """
    Removes unimportant regions from images based on LIME masks.

    Parameters:
    - images (list or np.ndarray): List or array of original images.
    - lime_masks (list or np.ndarray): List or array of LIME masks (1 for important regions, 0 for unimportant).
    - method (str): Method to fill unimportant regions ('blur', 'mean', or 'zero').
    - blur_sigma (int): Sigma for Gaussian blur if 'blur' method is selected.
    - save (bool): Whether to save processed images.
    - save_path (str): Directory path to save images if save is True.

    Returns:
    - processed_images (list): List of images with unimportant regions removed.
    """
    
    # Verify input types
    if not isinstance(images, (list, np.ndarray)):
        raise TypeError("Images should be a list or numpy array of image arrays.")
        
    if not isinstance(lime_masks, (list, np.ndarray)):
        raise TypeError("LIME masks should be a list or numpy array of mask arrays.")
        
    if len(images) != len(lime_masks):
        raise ValueError("The number of images and LIME masks must be the same.")
    
    # Create save directory if needed
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)
    
    processed_images = []
    
    for idx, (image, lime_mask) in enumerate(zip(images, lime_masks)):
        # Ensure image and mask dimensions match
        if image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and LIME mask at index {idx} have mismatched dimensions.")
        
        # Copy the image for processing
        processed_image = image.copy()
        
        # Apply the chosen method to unimportant regions (where lime_mask == 0)
        if method == 'blur':
            blurred_image = gaussian(processed_image, sigma=blur_sigma)
            processed_image[lime_mask == 0] = blurred_image[lime_mask == 0]
        
        elif method == 'mean':
            mean_value = image.mean(axis=(0, 1), keepdims=True) if image.ndim == 3 else image.mean()
            processed_image[lime_mask == 0] = mean_value
        
        elif method == 'zero':
            processed_image[lime_mask == 0] = 0
        
        else:
            raise ValueError("Invalid method chosen. Available methods: 'blur', 'mean', 'zero'.")
        
        # Append processed image to the list
        processed_images.append(processed_image)
        
        # Save processed image if required
        if save:
            image_to_save = img_as_ubyte(processed_image)
            imsave(os.path.join(save_path, f'lime_filtered_image_{idx}.png'), image_to_save, check_contrast=False)
            print(f"Saved: {os.path.join(save_path, f'lime_filtered_image_{idx}.png')}")
    
    return processed_images


In [ ]:
masks__ = np.array(masks)

In [ ]:
from skimage.transform import resize

def resize_mask(mask, image_shape):
    return resize(mask, image_shape[:2], mode='reflect', anti_aliasing=True, preserve_range=True)


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image_neg[0].shape}")
print(f"Original Mask shape: {masks_neg[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks_neg, single_image_neg)]

# Process images with the resized masks
processed_images = apply_lime_mask(
    images=single_image_neg,
    lime_masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/VGG16/Lime/Negative'  # Directory to save images
)


In [ ]:
### Applying LIME on Negative

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_VGG16.predict(images)

single_image_pos = test_features__[0] 
single_image_pos.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks_pos = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image_pos = test_features__[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image_pos,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask_ = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks_pos.append(mask_)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image_pos[0].shape}")
print(f"Original Mask shape: {masks_pos[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks_pos, single_image_pos)]

# Process images with the resized masks
processed_images = apply_lime_mask(
    images=single_image_pos,
    lime_masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/VGG16/Lime/Positive'  # Directory to save images
)


In [ ]:
### Train on new data

In [ ]:
new_path___ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Lime/Positive/'
new_prefix__ = 'ct_covid'

new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Lime/Negative/'
new_prefix__ = 'non_ct_covid'

In [ ]:
ct_covid_features_df_lime =  processImages(new_path___,1)#1--> covid-19 positive
ct_noncovid_features_df_lime =  processImages(new_path__,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs_lime = [ct_noncovid_features_df_lime, ct_covid_features_df_lime]
cvd_imgs_dataset_lime = pd.concat(cvd_imgs_lime)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_lime_ = cvd_imgs_dataset_lime.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_lime_)

In [ ]:
cvd_imgs_dataset_lime_.shape

In [ ]:
input_data_lime_x = cvd_imgs_dataset_lime_.iloc[:,:-1].to_numpy().reshape((600,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime_['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime_['class_label'])
cvd_imgs_dataset_lime_
cvd_imgs_dataset_lime_ = pd.get_dummies(cvd_imgs_dataset_lime_, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_y = np.array(cvd_imgs_dataset_lime_[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_lime, test_features_lime, train_labels_lime, test_labels_lime = train_test_split(
    input_data_lime_x, output_label_lime_y, test_size=.20, random_state=42)

In [ ]:
modelss = VGG16()
modelss.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

modelss.fit(train_features_lime, train_labels_lime, epochs=10, batch_size=16)

In [ ]:
#### Finding the Intersection

In [ ]:
masker = shap.maskers.Image("blur(32,32)", test_features.shape)

In [ ]:
masked_images_as_lists = []
image_shape = (300, 300, 1)

for image in test_features:
    # Ensure the image has the correct shape
    if image.shape != image_shape:
        raise ValueError(f"Image shape {image.shape} does not match expected shape {image_shape}")

    # Initialize the masker for a single image
    masker = shap.maskers.Image("blur(32,32)", image_shape)

    # Apply the masker to each image
    masked_image_tuple = masker(image, np.zeros(image.shape[:-1]))  # Adjust the mask as needed
    
    # Extract the first element of the tuple (which is the masked image)
    masked_image = masked_image_tuple[0]
    
    # Convert the masked image to a list and store it
    masked_images_as_lists.append(masked_image.tolist())


In [ ]:
import numpy as np
import os
from skimage.io import imsave
from skimage.util import img_as_ubyte

import numpy as np
import os
from skimage.io import imsave
from skimage.util import img_as_ubyte

def find_intersection_and_save(images, shap_masks, lime_masks, save=False, save_path='dataset_intersection'):
    if len(images) != len(shap_masks) or len(images) != len(lime_masks):
        raise ValueError("The number of images, SHAP masks, and LIME masks must be the same.")
    
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)

    intersection_images = []

    for idx, (image, shap_mask, lime_mask) in enumerate(zip(images, shap_masks, lime_masks)):
        if image.shape[:2] != shap_mask.shape or image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and masks at index {idx} have mismatched dimensions.")
        
        shap_mask = shap_mask.astype(bool)
        lime_mask = lime_mask.astype(bool)
        
        intersection_mask = np.logical_and(shap_mask, lime_mask)
        print(f"Intersection count at index {idx}: {np.sum(intersection_mask)}")  # Debugging line
        
        intersection_image = np.copy(image)

        if intersection_image.ndim == 2:
            intersection_image = np.stack([intersection_image] * 3, axis=-1)

        overlay_image = np.copy(intersection_image)
        highlight_color = [255, 0, 0]

        overlay_image[intersection_mask] = highlight_color

        final_image = np.where(intersection_mask[:, :, np.newaxis], overlay_image, intersection_image)

        intersection_images.append(final_image)
        
        if save:
            image_to_save = img_as_ubyte(final_image)

            if len(image_to_save.shape) not in [2, 3]:
                raise ValueError(f"Unexpected image shape for saving: {image_to_save.shape}")

            imsave(os.path.join(save_path, f'intersection_image_{idx}.png'), image_to_save)
    
    return intersection_images



In [ ]:
masker_shap = np.array(masked_images_as_lists)

In [ ]:
masker_shap_ = masker_shap[:, 0, :, :]
len(masker_shap_[:300])

In [ ]:
test_features_ = test_features[:, :, :, 0]
len(test_features_[:300])

In [ ]:
mask.shape

In [ ]:
new_mask = np.repeat(mask[np.newaxis, :, :], 930, axis=0) 

In [ ]:
intersection_images = find_intersection_and_save(test_features_, masker_shap_, new_mask, save=True, save_path="/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Intersection/")

In [ ]:
#### Train on the intersection dataset

In [ ]:
new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG16/Intersection/'
new_prefix_ = 'ct_covid'

In [ ]:
ct_covid_intersection_features_df =  processImages(new_path__,1)
ct_covid_intersection_features_df.shape

In [ ]:
cvd_imgs_intersection = [ct_noncovid_features_df, ct_covid_intersection_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs_intersection)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_intersection = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_intersection)

In [ ]:
cvd_imgs_dataset_intersection.shape

In [ ]:
input_data_intersection_x = cvd_imgs_dataset_intersection.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_intersection['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_intersection['class_label'])
cvd_imgs_dataset_intersection
cvd_imgs_dataset_intersection = pd.get_dummies(cvd_imgs_dataset_intersection, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_intersection_y = np.array(cvd_imgs_dataset_intersection[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_intersection_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_intersection_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_intersection, test_features_intersection, train_labels_intersection, test_labels_intersection = train_test_split(
    input_data_intersection_x, output_label_intersection_y, test_size=.20, random_state=42)

In [ ]:
model_inter = VGG16()
model_inter.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model_inter.fit(train_features_intersection, train_labels_intersection, epochs=10, batch_size=16)

### Data Augmentation

In [ ]:
image_size = 200
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Normalization(),
        tf.keras.layers.Resizing(image_size, image_size),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(factor=0.02),
        tf.keras.layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(train_features)

### Building Model

In [ ]:
def CNN_Mod(model, num_hl, hl_list,hl_conv_activation, ol_activation, dropout_val, inputShape,filters_,kernel_size_,stride_poolSize):
    '''
    CNN_Mod(model, num_hl, hl_list,hl_conv_activation, ol_activation, dropout_val, inputShape,filters_,kernel_size_,stride_poolSize):
        model = CNN Model
        num_hl = number of hidden layers
        hl_list = list of hidden layers
        hl_activation = hidden layer activation function
        out_activation = output layer activation function
        dropout_val = Dropout value
        inputShape = shape of input layer
        filters_ = filter size of Conv layer
        kernel_size_ = size of kernel (x,x)
        strides_poolSize = strides and max_pool_szie
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    inputs = tf.keras.layers.Input(shape=inputShape)
    # Augment data.
    augmented = data_augmentation(inputs)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(augmented)
    conv_layer_1_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[0], activation = hl_conv_activation)(flatten_layer)
    dense_layers = tf.keras.layers.Dropout(dropout_val[0])(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[1], activation = hl_conv_activation)(dense_layers)
    dense_layers = tf.keras.layers.Dropout(dropout_val[1])(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[2], activation = hl_conv_activation)(dense_layers)
    dense_layers = tf.keras.layers.Dropout(dropout_val[2])(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hl_list[3], activation = hl_conv_activation)(dense_layers)
    dense_layers = tf.keras.layers.Dropout(dropout_val[3])(dense_layers)
    
    logits = tf.keras.layers.Dense(units = 2, activation = ol_activation)(dense_layers)
    
    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs,outputs=logits)
    
    return model


def compile_model(model, loss_, learningRate, metrics_):
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest [mse', 'mae'] #since a regression model
    '''
    model.compile(tf.keras.optimizers.Adam(learning_rate=learningRate,weight_decay=0.001),loss=loss_,metrics=metrics_)
    return model

def buildModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
  '''
  def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
  '''
  history = model.fit(
    train_features, 
    train_labels,
    validation_split = val_split_size,
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )
  
  return history

def evaluateModel_loss(history):
    print()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Training loss','Validation loss'], loc = 'upper left')
    plt.savefig("Training_validation_loss_with_i.jpg")
    plt.show()
    plt.close()
    #plt.show()
    print()
def evaluateModel_accuracy(history):
    print()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epochs')
    plt.legend(['Training Accuracy','Validation Accuracy'], loc = 'upper left')
    #plt.show()
    plt.savefig("Training_validation_accuracy_with_i.jpg")
    plt.show()
    plt.close()
    print()

print(CNN_Mod.__doc__)

In [ ]:
# initiallizing and building the model
#cnn_1  = tf.keras.models.Sequential()

In [ ]:
'''
Hyperparameters:
Hyperparameters:
Convolution Layers filter Count: : 100
Convolution Layers Kernel Size: : 5
Max-pooling_Stride_ Size: : 2
Hiddel Layer 1 Units: 2884
dropout1: False
Hiddel Layer 2 Units: 1288
dropout2: True
Hiddel Layer 3 Units: 332
dropout3: False
Hiddel Layer 4 Units: 116
dropout4: False
lr: 0.0015272304174499124
Score: 0.9326164722442627
'''

In [ ]:
# initiallizing and building the model
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
cnn_1  = tf.keras.models.Sequential()
cnn_1  = CNN_Mod(cnn_1, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_1  = compile_model(cnn_1,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
history = buildModel(cnn_1,0.20,128,100,10, "val_loss", "min")

In [ ]:
evaluateModel_loss(history)

In [ ]:
evaluateModel_accuracy(history)

In [ ]:
cnn_1.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(cnn_1, to_file='cnn_1_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
cnn_1.save('cvd_cnn_1.h5')

In [ ]:
cnn_1.save_weights('cvd_cnn_weight_1.h5')

#### Testing Model

In [ ]:
label_predicted = cnn_1.predict((test_features))

In [ ]:
n_samples = 10

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = test_features[index].reshape(300, 300)
    plt.imshow(sample_image, cmap="binary")
    plt.title("Label:" + str(test_labels[index].argmax()))
    plt.axis("off")

plt.show()

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    #sample_image = image_predicted[index].reshape(300, 300)
    #plt.imshow(sample_image, cmap="binary")
    plt.title("Predicted:" + str(np.argmax(label_predicted[index])))
    plt.axis("off")

plt.show()

In [ ]:
def getCorrectLabel(x):
        if x == 0:
            return 'Non-Covid'
        if x == 1:
            return 'Covid'
        
def D1_to_D2(y_pred):
    y_true = {'y_true':[i[0] for i in y_pred]}
    y_true_df = pd.DataFrame(y_true)
    y_true_df = pd.get_dummies(y_true_df, columns =['y_true'])
    output_label_y = np.array(y_true_df[y_true_df.columns])
    return output_label_y

def getModelEvaluation(model, test_features_,test_labels_):
    predicted_labels = model.predict(np.stack(test_features_))
    df = pd.DataFrame(predicted_labels)
    df['Predicted_Labels'] = np.array(df.iloc[:,:2]).argmax(axis =1)
    df['Predicted_Labels'] = df['Predicted_Labels'].apply(getCorrectLabel)
    if (test_labels_.shape[1]==2):
        df['Actual_Labels'] = test_labels_[:,:2].argmax(axis = 1)
    else:
        df['Actual_Labels'] = D1_to_D2(test_labels_)[:,:2].argmax(axis = 1)
    df['Actual_Labels'] = df['Actual_Labels'].apply(getCorrectLabel)
    # df['Probality'] = np.max(np.array(df.iloc[:,:7]))
    #df.drop([0,1,2,3,4,5,6,7,8,9],axis =1,inplace=True)
    #print(df.head(100))
    cm = pd.crosstab(df.Predicted_Labels, df.Actual_Labels)
    fig = plt.figure(figsize=(20,5))
    ax1 = plt.subplot(121)
    sns.heatmap(cm,annot = True,cmap='Blues')
    ax1.set_title('')
    # Saving the figure.
    plt.savefig("test_Confusion_Matrix_with_i.jpg")
    plt.show()
    plt.close()

    pred = model.evaluate(test_features_,test_labels_)
    print("loss = " + str(pred[0]))
    print("test accuracy = " + str(pred[1]))

In [ ]:
import seaborn as sns
getModelEvaluation(cnn_1,test_features,test_labels)

### Testing the Trained Model on a different Dataset
- https://www.kaggle.com/datasets/luisblanche/covidct

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))

path1 = '/home/rkannan/Desktop/Cvd19_Classification/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/rkannan/Desktop/Cvd19_Classification/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

def processImage(imgDirPath,binary_label):
    img_names = list()
    with os.scandir(imgDirPath) as dirs:
        for entry in dirs:
            img_names.append(entry.name)
    #Creating features for images
    all_features = []
    for img in img_names:
        path = imgDirPath + img
        cv_img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)
        #create features for machine learning
        nFeatures = (cv_img2.shape[0]*cv_img2.shape[1])
        features = np.reshape(cv_img2, nFeatures)
        all_features.append(features)
        #print(features.shape)
    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]),dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]),dtype=int)
    return imgs_df


ct_covid_features_df =  processImage(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImage(path2,0)#0 ---> covnid-19 negative
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
display(cvd_imgs_dataset)

input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((746,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset_ = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset_[['output_encode_0','output_encode_1']])
#output_label_y = np.array(cvd_imgs_dataset[['output_encode']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))
#print('Output_y Data Shape for evaluation: \n{0}'.format(output_label_y_eval.shape))

In [ ]:
y_test_new = output_label_y
x_test_new = input_data_x

### Loading Trained Model on old dataset

In [ ]:
cnn_old  = tf.keras.models.Sequential()
cnn_old  = CNN_Mod(cnn_old, 4, [288, 128, 33, 11],'relu','softmax',[0.00,0.35,0.00,0.00],(300,300,1),100,5,2)
#compile and run model here
cnn_old  = compile_model(cnn_old,'categorical_crossentropy',0.0015272304174499124,['accuracy'])
cnn_old.load_weights('cvd_cnn_weight_1.h5')

In [ ]:
getModelEvaluation(cnn_old,x_test_new,y_test_new)

## Hyper Parameter Tuning

In [ ]:
import keras_tuner as kt
def KT_hp_build(hp,shape = (300,300,1), dropout_rate = 0.35):
    #initiallizing the model
    cnn = tf.keras.models.Sequential()
    
    filters_ = hp.Int("Convolution Layers filter Count: ",min_value = 100, max_value = 130, step = 16 )
    kernel_size_ = hp.Int("Convolution Layers Kernel Size: ",min_value = 5, max_value = 10 ,step = 3)
    strides_poolSize = hp.Choice("Max-pooling_Stride_ Size: ",[2])
    hl_conv_activation = 'relu'
    ol_activation = 'softmax'
    
    inputs = tf.keras.layers.Input(shape=shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_1 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(augmented)
    conv_layer_1_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_1)
    x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    #connnnvolultion layer 1 with Map-pooling
    conv_layer_2 = tf.keras.layers.Conv2D(filters = filters_,kernel_size = kernel_size_,strides = stride_poolSize, activation = hl_conv_activation)(x1)
    conv_layer_2_max_pool = tf.keras.layers.MaxPool2D(strides=stride_poolSize)(conv_layer_2)
    x2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(conv_layer_1_max_pool)
    
    flatten_layer = tf.keras.layers.Flatten()(x2)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 1 Units",min_value=2500, max_value=3000,step = 16), activation = hl_conv_activation)(flatten_layer)
    if hp.Boolean("dropout1"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 2 Units",min_value=1000, max_value=1500,step = 32), activation = hl_conv_activation)(dense_layers)
    if hp.Boolean("dropout2"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 3 Units",min_value=300, max_value=500,step = 32), activation = hl_conv_activation)(dense_layers)
    if hp.Boolean("dropout3"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    dense_layers = tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 4 Units",min_value=100, max_value=150,step = 4), activation = hl_conv_activation)(dense_layers)
    if hp.Boolean("dropout4"):
        dense_layers = tf.keras.layers.Dropout(dropout_rate)(dense_layers)
    
    logits = tf.keras.layers.Dense(units = 2, activation = ol_activation)(dense_layers)
    
    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs,outputs=logits)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=0.001, max_value=0.002, sampling="log")
    
    cnn.compile(
      #optimizer = 'adam', 
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss = 'categorical_crossentropy', 
      metrics = ['accuracy'])
    return model
    
#     # First Convolution Layer with Max-Pooling
#     cnn.add(tf.keras.layers.Conv2D(filters = filters_val,kernel_size = kernel_size_val,strides = strides_poolSize, activation ='relu',input_shape = shape))
#     cnn.add(tf.keras.layers.MaxPool2D(pool_size=strides_poolSize, strides=strides_poolSize))
    
#     # second Convolution Layer with Max-Pooling
#     cnn.add(tf.keras.layers.Conv2D(filters = filters_val,kernel_size = kernel_size_val,strides = strides_poolSize, activation ='relu'))
#     cnn.add(tf.keras.layers.MaxPool2D(pool_size=strides_poolSize, strides=strides_poolSize))
    
#     # Third Convolution Layer with Max-Pooling
#     cnn.add(tf.keras.layers.Conv2D(filters = filters_val,kernel_size = kernel_size_val,strides = strides_poolSize, activation ='relu'))
#     cnn.add(tf.keras.layers.MaxPool2D(pool_size=strides_poolSize, strides=strides_poolSize))
    
#     #Flattening
#     cnn.add(tf.keras.layers.Flatten())
    
#     #Full Connection
#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 1 Units",min_value=2500, max_value=3000,step = 16), activation = hl_conv_activation))
#     if hp.Boolean("dropout1"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))

#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 2 Units",min_value=1000, max_value=1500,step = 32), activation = hl_conv_activation))
#     if hp.Boolean("dropout2"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))

#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 3 Units",min_value=300, max_value=500,step = 32), activation = hl_conv_activation))
#     if hp.Boolean("dropout3"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))
    
#     cnn.add(tf.keras.layers.Dense(units = hp.Int("Hiddel Layer 4 Units",min_value=100, max_value=150,step = 4), activation = hl_conv_activation))
#     if hp.Boolean("dropout4"):
#         cnn.add(tf.keras.layers.Dropout(dropout_rate))
    
#     #output
#     cnn.add(tf.keras.layers.Dense(units = 2, activation = ol_activation))
    
    
#     # Define the optimizer learning rate as a hyperparameter.
#     learning_rate = hp.Float("lr", min_value=0.001, max_value=0.002, sampling="log")
    
#     cnn.compile(
#       #optimizer = 'adam', 
#       optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
#       loss = 'categorical_crossentropy', 
#       metrics = ['accuracy'])
#     return cnn

In [ ]:
KT_hp_build(kt.HyperParameters())

In [ ]:
#initialize the tuner
tuner = kt.RandomSearch(
    hypermodel= KT_hp_build,
    objective="val_accuracy", 
    overwrite = True,# Do not resume the previous search in the same directory.
    max_trials= 5,
    directory = "models/param_tuning",  # Set a directory to store the intermediate results.
    project_name= "param_tuning"
)

In [ ]:
tuner.search_space_summary() #print a summary of the search space

In [ ]:
#Visuallizing the hyperparameter tuning process
tuner.search(train_features, train_labels, 
             validation_data=(test_features, test_labels),
             validation_split=0.20,
             batch_size = 128,
             epochs = 100,
             # Use the TensorBoard callback.
             # The logs will be write to "models/tb_logs".
             callbacks=[
                tf.keras.callbacks.EarlyStopping(monitor="loss",patience=10,verbose=1,mode="min")]
                #tf.keras.callbacks.TensorBoard("models/tb_logs")],
             )

In [ ]:
tuner.results_summary()